<a href="https://colab.research.google.com/github/jrojasquiroz/msprov/blob/v3/2-identifica_ciudades.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identificación de ciudades principales

#### ¿Qué hacemos en este cuaderno?
Como los datos del [NOTI-SINADEF] están desagregados hasta el nivel distrital, necesitamos saber qué distritos forman parte de cada una de las *ciudades principales* para luego agregarlos y poder analizar los fallecimientos por COVID-19 a nivel de ciudades.

Para ello, en este cuaderno limpiaremos la cartografía de INEI que contiene los límites de cada *ciudad principal*.

¿Por qué? Porque los límites marcados por INEI a veces toman partes muy pequeñas de distritos en los que existen otras ciudades.

#### ¿Qué procedimiento seguimos?
Para hacerlo, (1) 'partimos' las manchas urbanas por distritos (en este cuaderno, a cada partición les llamaremos 'pedacitos'), (2) calculamos cuánta población vive en cada 'pedacito' de mancha urbana, y (3) calculamos cuánta población vive en toda la mancha urbana. Con estos tres datos podremos tener dos columnas que nos servirán para hacer el filtro de limpiado: el porcentaje poblacional que representa cada 'pedacito' de la mancha urbana a la que pertenece (*PR*), y el porcentaje poblacional que representa cada 'pedacito' del distrito al que pertenece (*PR_2*).

Cuando consigo estas dos variables, de manera manual reviso en QGIS algunos 'pedacitos' para ver qué *PR* y *PR_2* tienen. Luego de revisar algunas manchas urbanas veo que los 'pedacitos' que deberían irse (por ser muy pequeños) tienen un *PR* menor a 3.17%.

Sin embargo, veo que con ese filtro aplicado aún hay errores en algunas manchas urbanas. Por tanto, añado el umbral de *PR_2* menor a 13.15%, también después de una revisión visual de aquellos 'pedacitos' que creo deben mantenerse en el geodataframe.

#### ¿Qué obtenemos?
Al final de este proceso podremos saber (1) qué distritos forman parte de cada *ciudad principal* y (2) qué población tiene cada *ciudad principal*. Esta info se presenta en el archivo *ciudades_final.parquet*.

#### Datos utilizados
Para lograrlo utilizaremos tres geodataframes obtenidos del procesamiento en el cuaderno anterior:
1. Un gdf que tiene partidas las manchas urbanas por distritos y la población que la conforma (*ciudades_pob.parquet*). No nos interesa su geometría, sino sus datos.
2. Un gdf que tiene solo las manchas urbanas repartidas por distritos (*ciudades_interseccion.parquet*). Aquí si nos interesa su geometría.
3. Un df con la población de cada distrito del país, obtenida directamente de INEI (*pobxdist.xlsx*).

[NOTI-SINADEF]:https://www.datosabiertos.gob.pe/dataset/fallecidos-por-covid-19-ministerio-de-salud-minsa

In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os

In [4]:
os.chdir('/content/drive/MyDrive/Documentos personales/1-Investigaciones y análisis/Perú una país de provincias/revisión 1')

In [5]:
#1.Calcularemos cuánta población tiene cada 'pedacito' de las manchas urbanas
ciudades=gpd.read_parquet("data/ciudades_pob.parquet")
ciudades_pedacitos=ciudades.loc[:,['ID','Población']]
ciudades_pedacitos=ciudades_pedacitos.groupby("ID").sum()
ciudades_pedacitos.head()

,Población
ID,
1.0,8082.0
2.0,58094.0
3.0,10448.0
4.0,10663.0
5.0,33397.0


In [6]:
#2.Ahora cuánta población tiene toda la mancha urbana que conforman las ciudades.
ciudades_enteras=ciudades.loc[:,['CIUDAD','Población']]
ciudades_enteras=ciudades_enteras.groupby('CIUDAD').sum()
ciudades_enteras.head()

,Población
CIUDAD,
ABANCAY,66176.0
ANDAHUAYLAS,54508.0
AREQUIPA,843359.0
AYACUCHO,194200.0
AYAVIRI,18305.0


In [7]:
#3.Importamos el gdf con las manchas urbanas de las Ciudades Principales repartidas
#por distrito
ciudades_i=gpd.read_parquet("data/ciudades_interseccion.parquet")
ciudades_i.head()

,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,geometry,ID
0,ABANCAY,TAMBURCO,30109,0301,ABANCAY,"MULTIPOLYGON (((-8112020.618 -1529196.326, -81...",1
1,ABANCAY,ABANCAY,30101,0301,ABANCAY,"MULTIPOLYGON (((-8112178.097 -1531791.118, -81...",2
2,ANDAHUAYLAS,TALAVERA,30216,0302,ANDAHUAYLAS,"POLYGON ((-8176551.463 -1532941.437, -8176530....",3
3,ANDAHUAYLAS,SAN JERONIMO,30213,0302,ANDAHUAYLAS,"POLYGON ((-8167720.999 -1534466.258, -8167729....",4
4,ANDAHUAYLAS,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,"MULTIPOLYGON (((-8172249.289 -1534344.842, -81...",5


In [8]:
#4.Le añadimos los cálculos hechos previamente
#4.1. Primero, la población de toda la Mancha Urbana (PobMU)
ciudades_nuevo = gpd.GeoDataFrame(ciudades_i.merge(ciudades_enteras, on='CIUDAD', how='left'))
ciudades_nuevo.rename(columns={'Población':'PobMU'},inplace=True)

#4.2. Luego, la población solo del 'pedacito' de mancha urbana (PobParcial)
ciudades_nuevo = gpd.GeoDataFrame(ciudades_nuevo.merge(ciudades_pedacitos, on='ID', how='left'))
ciudades_nuevo.rename(columns={'Población':'PobParcial'},inplace=True)

#4.3.Esto nos permitirá saber qué porcentaje de población hay en cada 'pedacito'
#respecto del total de población de la Mancha Urbana de las Ciudades Principales
ciudades_nuevo['PR']= ciudades_nuevo['PobParcial']/ciudades_nuevo['PobMU']*100

ciudades_nuevo.head()

,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,geometry,ID,PobMU,PobParcial,PR
0,ABANCAY,TAMBURCO,30109,0301,ABANCAY,"MULTIPOLYGON (((-8112020.618 -1529196.326, -81...",1,66176.0,8082.0,12.212887
1,ABANCAY,ABANCAY,30101,0301,ABANCAY,"MULTIPOLYGON (((-8112178.097 -1531791.118, -81...",2,66176.0,58094.0,87.787113
2,ANDAHUAYLAS,TALAVERA,30216,0302,ANDAHUAYLAS,"POLYGON ((-8176551.463 -1532941.437, -8176530....",3,54508.0,10448.0,19.167829
3,ANDAHUAYLAS,SAN JERONIMO,30213,0302,ANDAHUAYLAS,"POLYGON ((-8167720.999 -1534466.258, -8167729....",4,54508.0,10663.0,19.562266
4,ANDAHUAYLAS,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,"MULTIPOLYGON (((-8172249.289 -1534344.842, -81...",5,54508.0,33397.0,61.269905


In [10]:
#5.Importamos la info de la población por distritos
dist_p = pd.read_excel("rawdata/pobxdist.xlsx")
dist_p.head()

,CODUBIGEO,POBDIST_2017
0,10101,32026
1,10102,145
2,10103,354
3,10104,449
4,10105,188


In [11]:
#6.Unimos el df obtenido en el paso 4 con el obtenido en el paso 5
ciudades_nuevo=gpd.GeoDataFrame(ciudades_nuevo.merge(dist_p,on='CODUBIGEO'))
ciudades_nuevo = ciudades_nuevo.fillna(0)

In [12]:
ciudades_nuevo.head()

,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,geometry,ID,PobMU,PobParcial,PR,POBDIST_2017
0,ABANCAY,TAMBURCO,30109,0301,ABANCAY,"MULTIPOLYGON (((-8112020.618 -1529196.326, -81...",1,66176.0,8082.0,12.212887,9171
1,ABANCAY,ABANCAY,30101,0301,ABANCAY,"MULTIPOLYGON (((-8112178.097 -1531791.118, -81...",2,66176.0,58094.0,87.787113,63106
2,ANDAHUAYLAS,TALAVERA,30216,0302,ANDAHUAYLAS,"POLYGON ((-8176551.463 -1532941.437, -8176530....",3,54508.0,10448.0,19.167829,11691
3,ANDAHUAYLAS,SAN JERONIMO,30213,0302,ANDAHUAYLAS,"POLYGON ((-8167720.999 -1534466.258, -8167729....",4,54508.0,10663.0,19.562266,12378
4,ANDAHUAYLAS,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,"MULTIPOLYGON (((-8172249.289 -1534344.842, -81...",5,54508.0,33397.0,61.269905,34896


In [13]:
#7.Calculamos qué porcentaje representa la población de cada 'pedacito' de mancha urbana
#sobre el total de la población del distrito en el que se inserta.
ciudades_nuevo['PR_2']=ciudades_nuevo['PobParcial']/ciudades_nuevo['POBDIST_2017']*100
ciudades_nuevo.head()

,CIUDAD,NOMBDIST,CODUBIGEO,IDPROV,NOMBPROV,geometry,ID,PobMU,PobParcial,PR,POBDIST_2017,PR_2
0,ABANCAY,TAMBURCO,30109,0301,ABANCAY,"MULTIPOLYGON (((-8112020.618 -1529196.326, -81...",1,66176.0,8082.0,12.212887,9171,88.125613
1,ABANCAY,ABANCAY,30101,0301,ABANCAY,"MULTIPOLYGON (((-8112178.097 -1531791.118, -81...",2,66176.0,58094.0,87.787113,63106,92.057807
2,ANDAHUAYLAS,TALAVERA,30216,0302,ANDAHUAYLAS,"POLYGON ((-8176551.463 -1532941.437, -8176530....",3,54508.0,10448.0,19.167829,11691,89.367890
3,ANDAHUAYLAS,SAN JERONIMO,30213,0302,ANDAHUAYLAS,"POLYGON ((-8167720.999 -1534466.258, -8167729....",4,54508.0,10663.0,19.562266,12378,86.144773
4,ANDAHUAYLAS,ANDAHUAYLAS,30201,0302,ANDAHUAYLAS,"MULTIPOLYGON (((-8172249.289 -1534344.842, -81...",5,54508.0,33397.0,61.269905,34896,95.704379


Exportamos para hacer una exploración visual de lo obtenido.

In [14]:
# Ruta del archivo GeoParquet comprimido de salida
output_parquet = 'data/ciudades_ok.parquet'

# Guardar como GeoParquet con compresión gzip
ciudades_nuevo.to_parquet(output_parquet, compression='gzip', index=False)

In [15]:
#8.¿Qué valores excluir? Puedo ir probando por percentiles
percentil=ciudades_nuevo['PR']
percentil = percentil.dropna(how='all')
np.percentile(percentil,q=[35]) #El P35 creo que es el más adecuado.

array([3.16922066])

In [21]:
#8.¿Qué valores excluir? Puedo ir probando por percentiles
percentil=ciudades_nuevo['PR_2']
percentil = percentil.dropna(how='all')
np.percentile(percentil,q=[30]) #El P30 creo que es el más adecuado.

array([12.20556866])

Con el geodataframe obtenido (*ciudades_ok.parquet*), después de analizar visualmente la información, creo que puede utilizarse un criterio de filtro que tome en cuenta los valores por encima del percentil 35 de la columna 'PR' (% poblacional que representa el 'pedacito' respecto del total de la mancha urbana) y por encima del percentil 30 de la columna 'PR_2' (% poblacional que representa el 'pedacito' del total del distrito en el que se inserta).

In [22]:
gdf_final=ciudades_nuevo[(ciudades_nuevo['PR']>=3.16922066) | (ciudades_nuevo['PR_2']>=13.15)]

Lo que me inicia convenciendo sobre la elección de estos porcentajes son los casos de los distritos de Piura y Catacaos, y el de La Merced y San Ramón.

En el caso de PR_2, si bien el percentil se veía bueno, generaba un problema con la ciudad de Trujillo y Laredo, que comparten el distrito de Trujillo; por ello se decide usar el 13.15 como límite, que se ajusta a los números de estas dos ciudades.

In [23]:
#1.Filtro para centrarme en las columnas CIUDAD y UBIGEO
filtro=gdf_final
filtro=filtro[['CIUDAD','CODUBIGEO']]
filtro.head()

,CIUDAD,CODUBIGEO
0,ABANCAY,30109
1,ABANCAY,30101
2,ANDAHUAYLAS,30216
3,ANDAHUAYLAS,30213
4,ANDAHUAYLAS,30201


In [24]:
#2. Encontramos las ciudades que comparten distritos.
filtro2=filtro[filtro['CODUBIGEO'].duplicated(keep=False)]
filtro2

,CIUDAD,CODUBIGEO
167,PISCO,110501
169,TUPAC AMARU,110501


En el caso de Pisco y Túpac Amaru, se trata de dos ciudades que acumulan 3 distritos. La delimitación sugiere que en estas ciudades se concentra la mayor parte de los tres distritos. Por tanto considero que, para no excluirlas del análisis, pueden considerarse como una sola ciudad.

In [25]:
#3. Reemplazamos TUPAC AMARU por PISCO, con la finalidad de que a partir de ahora sean consideradas
#como una sola ciudad
#3.1.Primero me fijo que no haya otra ciudad TUPAC AMARU, para eso veo su frecuencia. Debería ser 2 (porque
#está en dos distritos).
freq = filtro.groupby(['CIUDAD']).count()
print(freq)

             CODUBIGEO
CIUDAD                
ABANCAY              2
ANDAHUAYLAS          3
AREQUIPA            17
AYACUCHO             5
AYAVIRI              1
...                ...
TUPAC AMARU          2
VIRU                 1
YAURI                1
YURIMAGUAS           1
ZARUMILLA            2

[91 rows x 1 columns]


In [26]:
gdf_final['CIUDAD']=gdf_final['CIUDAD'].replace(['TUPAC AMARU'], 'PISCO')

/usr/local/lib/python3.10/dist-packages/geopandas/geodataframe.py:1528: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [27]:
freq = gdf_final.groupby(['CIUDAD']).count()
print(freq) #Ya no aparece TUPAC AMARU

             NOMBDIST  CODUBIGEO  IDPROV  NOMBPROV  geometry  ID  PobMU  \
CIUDAD                                                                    
ABANCAY             2          2       2         2         2   2      2   
ANDAHUAYLAS         3          3       3         3         3   3      3   
AREQUIPA           17         17      17        17        17  17     17   
AYACUCHO            5          5       5         5         5   5      5   
AYAVIRI             1          1       1         1         1   1      1   
...               ...        ...     ...       ...       ...  ..    ...   
TUMBES              1          1       1         1         1   1      1   
VIRU                1          1       1         1         1   1      1   
YAURI               1          1       1         1         1   1      1   
YURIMAGUAS          1          1       1         1         1   1      1   
ZARUMILLA           2          2       2         2         2   2      2   

             PobParcial 

In [28]:
#4. Como paso final, vamos a eliminar duplicados.
#Pasa que como TUPAC AMARU y PISCO compartían un distrito (PISCO, UBIGEO 110501), ese distrito va a salir duplicado
#y nos puede causar problemas más adelante.

#4.1.Primero comprobamos qué largo tiene este gdf
len(gdf_final)

196

In [29]:
#4.2.Ahora sí excluimos el duplicado
gdf_final2=gdf_final.drop_duplicates(['CODUBIGEO'])
len(gdf_final2)

195

In [30]:
#4.3.Exportamos
output_parquet = 'data/ciudades_final.parquet'
gdf_final2.to_parquet(output_parquet, compression='gzip', index=False)

gdf_final2.to_excel("data/ciudades_final.xlsx") #para trabajar en R